# IBM Data Science Professional Certificate Captsone Project
#### Hello and welcome! :)
This notebook is part of my capstone project of the certificate on Coursera called IBM Data Science Professional Certificate. <br>If you are interested about this course, you can get more information by clicking [here](https://www.coursera.org/professional-certificates/ibm-data-science?).<br><br>
In this notebook I will showcase some of the things I learned during my lessons.
The assignment consists of three main sections: <br>i) Scrapping a website (Wikipedia). <br>ii) Cleaning the data obtained. <br>iii) Perform a cluster analysis with a visualization.

## Part I: Scrapping a Wikipedia page
In this section, we will scrap a Wikipedia page with information concerning Canadian postal codes.

The first thing we need to do is to install beautiful soup, the Python package that allows us to crawl websites.

In [207]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [208]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [209]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [210]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


Then we import the necessary packages

In [211]:
from bs4 import BeautifulSoup

In [212]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

In [213]:
#Then we indicate the irl we will be parsing from
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#We also have to create a handle, page, to handle the contents of the website
page = requests.get(url)
#We will store the contents of the website under doc
doc = lh.fromstring(page.content)
#We will parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

For sanity check, ensure that all the rows have the same width. If not, we probably got something more than just the table.
Check the length of the first 12 rows

In [214]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

We start by parsing the first row as our header.

In [215]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



Then we create a a Pandas dataframe

In [216]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Just to be sure, let’s check the length of each column. Ideally, they should all be the same.

In [217]:
[len(C) for (title,C) in col]

[287, 287, 287]

Now we are ready to create the DataFrame.

In [218]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [219]:
df.head(5)

,Postcode,Borough,Neighbourhood\n
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


We remove the last \n characters of each neighbourhood field

In [220]:
df['Neighbourhood\n'] = df['Neighbourhood\n'].str.rstrip('\n')
df.head()

,Postcode,Borough,Neighbourhood\n
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


We correct the names of the columns

In [221]:
df=df.rename(columns = {'Neighbourhood\n':'Neighbourhood'})
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


We remove the boroughs "Not assigned"

In [222]:
df = df[df.Borough != "Not assigned"]

In [223]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


We merge rows with same postcode

In [224]:
foo = lambda a: ", ".join(a) 
df = df.groupby(['Postcode','Borough']).agg({'Neighbourhood': foo}).reset_index()

We look for the rows with a non-assigned neighbourhood

In [225]:
print(df.loc[df['Neighbourhood'] == 'Not assigned'])

   Postcode       Borough Neighbourhood
93      M9A  Queen's Park  Not assigned


We make the necessary amendments

In [226]:
df.loc[93, 'Neighbourhood'] = "Queen's Park"
df.loc[93, 'Neighbourhood']

"Queen's Park"

We verify that the changes have been made and there are no instances of "Not assigned" in the neighbourhood column

In [227]:
print(df.loc[df['Neighbourhood'] == 'Not assigned'])

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


We check the shape of our dataframe

In [228]:
df.shape

(103, 3)

Here is our final dataframe!

In [229]:
df.reset_index(drop=True)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## Part II: Scrapping a Wikipedia page
##### In this section, we will include the geospatial coordinates of each postal code to the dataframe

We start by opening the file with the geospatial coordinates (that we downloaded directly from the Coursera page)

In [230]:
Coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
Coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We rename the first column

In [231]:
Coordinates_df = Coordinates_df.rename(columns = {'Postal Code':'Postcode'})
Coordinates_df

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Then we merge the dataframes

In [232]:
merged_df = Coordinates_df.merge(df, how = 'inner', on = ['Postcode'])
merged_df.head()

,Postcode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


Then we change the order of the columns

In [233]:
df = merged_df[['Postcode','Borough','Neighbourhood','Latitude','Longitude']]
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Then we sort by latitude

In [234]:
df = df.sort_values(by =['Latitude'],ascending=True).reset_index(drop=True)
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
1,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321
2,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999
3,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
4,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
5,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
6,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819
7,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
8,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
9,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


## Part III: Explore and cluster the neighborhoods in Toronto
##### In this section, we will generate a map to visualize Toronto's neighborhoods and how they cluster together.

We start by installing the necessary packages

In [235]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [236]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd
import geopy

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


We ensure our dataframe is still working

In [237]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


Then we use geopy library to get the latitude and longitude values of Toronto.
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent toronto_explorer, as shown below.


In [238]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Then we create a map of Toronto using latitude and longitude values and putting neighbourhoods on top

In [239]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
Define Foursquare Credentials and Version

In [240]:
CLIENT_ID = 'M5LMD3A2CVW4JCHJWUIZOZT0LM3GZ3HTDB0RTJJHJIJXTT0V' # your Foursquare ID
CLIENT_SECRET = 'OITV0TXXBKGCTVMCFWON1UDAY5M2DTQ31ZBQEV13PN044SMG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M5LMD3A2CVW4JCHJWUIZOZT0LM3GZ3HTDB0RTJJHJIJXTT0V
CLIENT_SECRET:OITV0TXXBKGCTVMCFWON1UDAY5M2DTQ31ZBQEV13PN044SMG


Let's explore the first neighborhood in our dataframe.
Get the neighborhood's name.

In [241]:
df.loc[0, 'Neighbourhood']

'Alderwood, Long Branch'

Get the neighbourhood's latitude and longitude values.

In [242]:
neighbourhood_latitude = df.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = df.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = df.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Alderwood, Long Branch are 43.60241370000001, -79.54348409999999.


Now, let's get the top 100 venues that are in Alderwood, Long Branch within a radius of 500 meters.
First, let's create the GET request URL. Name your URL url.

In [243]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
 
radius = 500 # define radius
 
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL



'https://api.foursquare.com/v2/venues/explore?&client_id=M5LMD3A2CVW4JCHJWUIZOZT0LM3GZ3HTDB0RTJJHJIJXTT0V&client_secret=OITV0TXXBKGCTVMCFWON1UDAY5M2DTQ31ZBQEV13PN044SMG&v=20180605&ll=43.60241370000001,-79.54348409999999&radius=500&limit=100'

Send the GET request and examine the results

In [244]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2be7d73907e70021217f5e'},
 'response': {'headerLocation': 'Alderwood',
  'headerFullLocation': 'Alderwood, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 43.60691370450001,
    'lng': -79.53728145264961},
   'sw': {'lat': 43.597913695500004, 'lng': -79.54968674735036}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c47927c1ddec928fbec9d32',
       'name': 'Il Paesano Pizzeria & Restaurant',
       'location': {'address': '396 Browns Line',
        'crossStreet': 'at Horner Ave',
        'lat': 43.60128,
        'lng': -79.545028,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.60128,
          'lng': -79.545028}],
        'distance': 177,
    

From the Foursquare lab in the previous module, we know that all the information is in the items key. 
Before we proceed, let's borrow the get_category_type function from the Foursquare lab.
Function that extracts the category of the venue

In [245]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [246]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Il Paesano Pizzeria & Restaurant,Pizza Place,43.601280,-79.545028
1,Toronto Gymnastics International,Gym,43.599832,-79.542924
2,Timothy's Pub,Pub,43.600165,-79.544699
3,Tim Hortons,Coffee Shop,43.602396,-79.545048
4,Pizza Pizza,Pizza Place,43.605340,-79.547252


And how many venues were returned by Foursquare?

In [247]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


Now we are going to explore all the neighbourhoods in Toronto.
Let's create a function to repeat the same process to all the neighborhoods in Toronto.

In [248]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [249]:
# type your answer here

toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )



Alderwood, Long Branch
Humber Bay Shores, Mimico South, New Toronto
Kingsway Park South West, Mimico NW, The Queensway West, Royal York South West, South of Bloor
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea
Brockton, Exhibition Place, Parkdale Village
Canada Post Gateway Processing Centre
Harbourfront East, Toronto Islands, Union Station
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Berczy Park
Stn A PO Boxes 25 The Esplanade
Design Exchange, Toronto Dominion Centre
Little Portugal, Trinity
Commerce Court, Victoria Hotel
First Canadian Place, Underground city
Parkdale, Roncesvalles
Adelaide, King, Richmond
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
St. James Town
Runnymede, Swansea
Chinatown, Grange Park, Kensington Market
The Kingsway, Montgomery 

Let's check the size of the resulting dataframe

In [250]:
print(toronto_venues.shape)
toronto_venues.head()

(2246, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Alderwood, Long Branch",43.602414,-79.543484,Il Paesano Pizzeria & Restaurant,43.601280,-79.545028,Pizza Place
1,"Alderwood, Long Branch",43.602414,-79.543484,Toronto Gymnastics International,43.599832,-79.542924,Gym
2,"Alderwood, Long Branch",43.602414,-79.543484,Timothy's Pub,43.600165,-79.544699,Pub
3,"Alderwood, Long Branch",43.602414,-79.543484,Tim Hortons,43.602396,-79.545048,Coffee Shop
4,"Alderwood, Long Branch",43.602414,-79.543484,Pizza Pizza,43.605340,-79.547252,Pizza Place


Let's check how many venues were returned for each neighborhood

In [251]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",4,4,4,4,4,4
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,58,58,58,58,58,58


Let's find out how many unique categories can be curated from all the returned venues

In [252]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


Now we will analyze each neighbourhood

In [253]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Let's examine the dataframe

In [254]:
toronto_onehot.shape

(2246, 270)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [255]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Summer Camp,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.010000,0.000000,0.030000,0.000

Let's confirm the new size

In [256]:
toronto_grouped.shape

(99, 270)

Let's print each neighborhood along with the top 5 most common venues

In [257]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.07
1              Café  0.04
2        Steakhouse  0.04
3               Bar  0.04
4  Asian Restaurant  0.03


----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1                     Lounge  0.25
2  Latin American Restaurant  0.25
3             Clothing Store  0.25
4          Accessories Store  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0                 Park  0.50
1           Playground  0.25
2          Coffee Shop  0.25
3   Mexican Restaurant  0.00
4  Monument / Landmark  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
            venue  freq
0   Grocery Store  0.18
1        Pharmacy  0.09
2     Video Store  0.09
3  Sandwich Place  0.09
4  Discount Store  0.09


----Alderwood, Long Branch----
                venue  freq
0         Pizza

Now we will put all this into a pandas dataframe.
First, let's write a function to sort the venues in descending order.

In [258]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [259]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Restaurant,Breakfast Spot,Thai Restaurant,Hotel,Asian Restaurant,Cosmetics Shop
1,Agincourt,Clothing Store,Breakfast Spot,Latin American Restaurant,Lounge,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Dog Run
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Coffee Shop,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pharmacy,Fried Chicken Joint,Beer Store,Japanese Restaurant,Sandwich Place,Discount Store,Fast Food Restaurant,Pizza Place,Video Store
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Coffee Shop,Sandwich Place,Athletics & Sports,Pub,Pool,Skating Rink,Gym,Eastern European Restaurant


Finally, we are going to perform our cluster analysis by neighbourhood.
We start by running the k-means methodology to cluster the neighborhood into 5 clusters (we choose this randomnly although in the final project we will optimize to see the best number of clusters needed).

In [260]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Then we create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [261]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,0.0,Pizza Place,Pharmacy,Coffee Shop,Sandwich Place,Athletics & Sports,Pub,Pool,Skating Rink,Gym,Eastern European Restaurant
1,M8V,Etobicoke,"Humber Bay Shores, Mimico South, New Toronto",43.605647,-79.501321,0.0,Café,Pharmacy,Bakery,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Pizza Place,Restaurant,Business Service,Gym
2,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen...",43.628841,-79.520999,0.0,Hardware Store,Bakery,Grocery Store,Fast Food Restaurant,Discount Store,Convenience Store,Sandwich Place,Burrito Place,Burger Joint,Social Club
3,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,0.0,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Bar,Coffee Shop,Sculpture Garden,Plane,Boat or Ferry
4,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509,0.0,Construction & Landscaping,Baseball Field,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant


We make the necessary amendments for the CLuster Format column to work in the next function (need to remove NA and convert into integer values)

In [262]:
toronto_merged ['Cluster Labels'] = toronto_merged ['Cluster Labels'].fillna(0)
toronto_merged ['Cluster Labels'] = toronto_merged ['Cluster Labels'].astype(int)

Finally, let's visualize the resulting clusters.

In [263]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

Cluster One

In [264]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Etobicoke,0,Pizza Place,Pharmacy,Coffee Shop,Sandwich Place,Athletics & Sports,Pub,Pool,Skating Rink,Gym,Eastern European Restaurant
1,Etobicoke,0,Café,Pharmacy,Bakery,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Pizza Place,Restaurant,Business Service,Gym
2,Etobicoke,0,Hardware Store,Bakery,Grocery Store,Fast Food Restaurant,Discount Store,Convenience Store,Sandwich Place,Burrito Place,Burger Joint,Social Club
3,Downtown Toronto,0,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Bar,Coffee Shop,Sculpture Garden,Plane,Boat or Ferry
4,Etobicoke,0,Construction & Landscaping,Baseball Field,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
5,West Toronto,0,Coffee Shop,Breakfast Spot,Café,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Gym,Bakery
6,Mississauga,0,Coffee Shop,Hotel,Burrito Place,Fried Chicken Joint,Sandwich Place,Middle Eastern Restaurant,American Restaurant,Gym,Mediterranean Restaurant,Dog Run
7,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant,Scenic Lookout,Fried Chicken Joint,Brewery,Restaurant,Baseball Stadium
8,Etobicoke,0,Park,Convenience Store,Liquor Store,Beer Store,Shopping Plaza,Cosmetics Shop,Coffee Shop,Pizza Place,Pet Store,Café
9,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Farmers Market,Café,Seafood Restaurant,Cheese Shop,Bakery,Beer Bar,Steakhouse,Shopping Mall


Cluster 2

In [265]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,York,1,Convenience Store,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
76,Scarborough,1,Playground,Convenience Store,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant


Cluster 3

In [266]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Etobicoke,2,Park,River,Pool,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
41,East York,2,Park,Convenience Store,Coffee Shop,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
43,Etobicoke,2,Pizza Place,Park,Sandwich Place,Mobile Phone Shop,Yoga Studio,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop
44,York,2,Park,Fast Food Restaurant,Women's Store,Market,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
45,Central Toronto,2,Park,Playground,Trail,Summer Camp,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
51,Central Toronto,2,Park,Bus Line,Sushi Restaurant,Trail,Jewelry Store,Dumpling Restaurant,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
62,North York,2,Park,Construction & Landscaping,Basketball Court,Bakery,Yoga Studio,Electronics Store,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
70,Central Toronto,2,Park,Lawyer,Swim School,Bus Line,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
73,North York,2,Park,Airport,Snack Place,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


Cluster 4

In [267]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,North York,3,Empanada Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


Cluster 5

In [268]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Scarborough,4,Bar,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Filipino Restaurant


# Thank you for reading this project!
I hope you enjoyed going through it as much as I did working on it!

This notebook is part of my final capstone project of the certificate on Coursera called IBM Data Science Professional Certificate. If you are interested about this course, you can get more information by clicking [here](https://www.coursera.org/professional-certificates/ibm-data-science?).